In [22]:
import os

In [23]:
%pwd

'/home/akill_sud/Documents/projects'

In [24]:
os.chdir("../")

In [25]:
%pwd

'/home/akill_sud/Documents'

In [26]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/mr-akillraja/Cauil_Disease_Prediction.mlflow"

In [27]:
import dagshub
dagshub.init(repo_owner='mr-akillraja', repo_name='Cauil_Disease_Prediction', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter npip install dagshubame', 'value')
  mlflow.log_metric('metric name', 1)

[2024-11-07 12:34:05,600 INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/mr-akillraja/Cauil_Disease_Prediction "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "mr-akillraja/Cauil_Disease_Prediction"

[2024-11-07 12:34:05,610 INFO: helpers: Initialized MLflow to track repo "mr-akillraja/Cauil_Disease_Prediction"]


Repository mr-akillraja/Cauil_Disease_Prediction initialized!

[2024-11-07 12:34:05,615 INFO: helpers: Repository mr-akillraja/Cauil_Disease_Prediction initialized!]
[2024-11-07 12:36:05,725 WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /mr-akillraja/Cauil_Disease_Prediction.mlflow/api/2.0/mlflow/runs/create]


In [28]:
import tensorflow as tf

model = tf.keras.models.load_model('artifacts/training/model.h5')

OSError: No file or directory found at artifacts/training/model.h5

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int

In [8]:
from Cauliflower_Classifier.constants import *
from Cauliflower_Classifier.utils.common import read_yaml_file, create_directories,save_json

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/dataset",
            mlflow_uri = "https://dagshub.com/mr-akillraja/Cauil_Disease_Prediction.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config
    

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self,):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss" : self.score[0],"accuracy":self.score[1]}
        save_json(path = Path("scores.json"),data = scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],'accuracy':self.score[1]}
            )
            if tracking_url_type_store != 'file':
                mlflow.keras.log_model(self.model, "model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")


In [12]:
try: 
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-07 11:01:46,928 INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-07 11:01:46,932 INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-07 11:01:46,933 INFO: common: Created directory at : artifacts]
Found 195 images belonging to 4 classes.


2024-11-07 11:01:48.708557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-11-07 11:01:51.208411: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-11-07 11:01:51.261457: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


 1/13 [=>............................] - ETA: 41s - loss: 0.1965 - accuracy: 1.0000

2024-11-07 11:01:52.196193: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-11-07 11:01:52.220351: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


 2/13 [===>..........................] - ETA: 8s - loss: 0.1977 - accuracy: 1.0000 

2024-11-07 11:01:52.995040: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


13/13 [==============================] - 17s 1s/step - loss: 1.5473 - accuracy: 0.2615
[2024-11-07 11:02:05,325 INFO: common: Json file saved at : scores.json]


2024/11/07 11:02:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-11-07 11:02:07,998 WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmptehg75tv/model/data/model/assets
[2024-11-07 11:02:08,584 INFO: builder_impl: Assets written to: /tmp/tmptehg75tv/model/data/model/assets]


/home/akill_sud/anaconda3/envs/cnncls/lib/python3.8/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/11/07 11:02:37 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
